# Sudoku as an Integer Linear Program

In [ ]:
using JuMP, GLPK

Once the packages are intialized we create our model using the GLPK optimizer and name it model.

Secondly we have created an individual variable for each and every instance of the row, column, and the number in that row column spot. So for i-th row, and j-th column there are 9 possible numbers. 

The last portion of our variable constructor says "Bin". This assigns a binary value to every i,j,k. This 1 will be used to determine if that value k is filling that i,j spot.

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer))

# Number of columns, rows, values

@variable(model, x[i=1:9, j=1:9, k=1:9], Bin)
# The syntax of this model is such that if any indices of x has a number k it will equal 1

The constraints below were written to follow the format of
\begin{equation*}
\sum_{n=1}^n x_{ijk} = 1
\end{equation*}
This will ensure for whatever combination of sum i's and sum j's only one k will be equal to 1 and that column or row will have one number. In other words no repeat numbers.

# Column Constraint

The syntax of this constraint is

@constraint ( name of Model(),

individual name of constraints (which i iterated) ,

expression i wanted my constraint to be (i.e. the formula from class)
)

So as before this column constraint is written as
\begin{equation}
\sum_{i=1}^9 x_{ijk} = 1\\ j=1:9\\k=1:9\\
\end{equation}
this implies that as you hold j constant you will iterate through every row for that column one unique k value. So we will do this for j's

In [ ]:
@constraint(model, col[j=1:9,k=1:9], sum(x[i,j,k] for i in 1:9 )==1)

JuMP allows you to write the Sudoku constraints and it will automatically add the constraints to a dictionary. I went ahead and set this and called one of the symbolic keys below.

In [ ]:
con_dict = model.obj_dict

In [ ]:
con_dict[:col][1]

# Row Constraint

\begin{equation}
\sum_{j=1}^9 x_{ijk} = 1, \\i=1:9,\\k=1:9\\
\end{equation}

In [ ]:
@constraint(model,row[i=1:9,k=1:9], sum(x[i,j,k] for j in 1:9 )==1)

# Completely Filled Sudoku Constraint

\begin{equation}
\sum_{k=1}^9 x_{ijk} = 1, \\i=1:9,\\j=1:9\\
\end{equation}

This simply says that every index has "some" value one through nine. In other words does the spot have an number? Yes, then x of ijk equals 1

In [ ]:
@constraint(model, filled[i=1:9,j=1:9], sum(x[i,j,k] for k in 1:9)==1)

# Sub Matrices  Sudoku Constraint

\begin{equation}
\sum_{j=mq-m+1}^{mq} \sum_{i=mp-m+1}^{mp} x_{ijk} = 1, \\k=1:9,\\p=1:m\\q=1:m\\
\end{equation}

Admittedly I had some trouble with this constraint and as I was under a time constraint I wrote this in brute force fashion. The constraints have their own individual names that outline their index position in the 2-dimensional array Sudoku.

In [ ]:
@constraint(model, submatUL[k=1:9], sum(sum(x[1:3,1:3,k]))==1)
@constraint(model, submatUM[k=1:9], sum(sum(x[1:3,4:6,k]))==1)
@constraint(model, submatUR[k=1:9], sum(sum(x[1:3,7:9,k]))==1)

@constraint(model, submatML[k=1:9], sum(sum(x[4:6,1:3,k]))==1)
@constraint(model, submatMM[k=1:9], sum(sum(x[4:6,4:6,k]))==1)
@constraint(model, submatMR[k=1:9], sum(sum(x[4:6,7:9,k]))==1)

@constraint(model, submatLL[k=1:9], sum(sum(x[7:9,1:3,k]))==1)
@constraint(model, submatLM[k=1:9], sum(sum(x[7:9,4:6,k]))==1)
@constraint(model, submatLR[k=1:9], sum(sum(x[7:9,7:9,k]))==1)

# Diagonal Constraint


This below is the Top to Down diagonal where every row and column share the same index value.
\begin{equation}
\sum_{i=1}^9 x_{ijk} = 1, \\j=1:9,\\k=1:9\\
\end{equation}

This below is the reverse index starting from the last row and the first column and ending in the first row and last column.
\begin{equation}
\sum_{i=1}^9 x_{ijk} = 1, \\j=9:1,\\k=1:9\\
\end{equation}

In [ ]:
row_DT = []
for i in 9:-1:1
    append!(row_DT,i)
end


col_DT = []
for j in 1:9
    append!(col_DT,j)
end

@constraint(model, diagonalDT[k=1:9], sum(sum(x[row_DT[i],col_DT[i],k] for i in 1:9))==1)

In [ ]:
@constraint(model, diagonalTD[k=1:9], sum(sum(x[i,i,k] for i in 1:9))==1)

In [ ]:
constraint_dict = model.obj_dict

In [ ]:
constraint_dict[:diagonalDT]

In [ ]:
initial_grid = [
    3 0 0 0 0 0 0 0 9;
    0 0 0 9 0 0 0 7 5;
    0 0 0 0 0 0 0 0 0;
    0 0 4 8 0 6 0 0 2;
    5 0 0 1 0 0 0 0 0;
    8 0 6 0 3 0 4 5 0;
    0 0 8 0 0 0 0 0 0;
    0 0 0 0 0 0 0 0 0;
    0 0 0 0 0 0 0 3 0;
]

sol_grid = initial_grid[:,:]

# Tuning on the 'x[i,j,k]' values from the intial given soduko.

This cell below will take the intial solution from the matrix above and turn those specific x[i,j,k] constraints on **AS** constraints

In [ ]:
for i=1:9
    for j=1:9
        if sol_grid[i,j]!=0
            @constraint(model, (x[i,j,sol_grid[i,j]])==1)
        end
    end
end

In [ ]:
optimize!(model)

This optimize command is straightforward it finds all feasible solutions and below I have written it so that for every feasible number in the vsariable set it extracts the k value and places it in the 0 or "empty" spaces in the Sudoku.

In [ ]:
turned_on = JuMP.value.(x)
sol = zeros(Int,9,9)
for i in 1:9
    for j in 1:9
        for k in 1:9
            if turned_on[i,j,k]==1
                sol[i,j]=k
            end
        end
    end
end
sol